In [2]:
import numpy as np

In [3]:
''' 
####################CLIFF WALKING ENVIRONMENT#########################

A schematic view of the environment-

o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
S  x  x  x  x  x  x  x  x  x  x  T

Actions: 
    UP (0)
    DOWN (1)
    RIGHT (2)
    LEFT (3)

Rewards: 
     0 for going in Terminal state
    -100 for falling in the cliff
    -1 for all other actions in any state

Note: State remains the same on going out of the maze (but -1 reward is given)
      The episode ends and the agent returns to the start state after falling in the cliff

'''
START_STATE = 36
TERMINAL_STATE = 47
def reward(state):
    if(state == TERMINAL_STATE):
        reward = 0
    elif(state > START_STATE and state < TERMINAL_STATE):
        reward = -100
    else:
        reward = -1
    return reward

def env(state, action):
    # return_val = [prob, next state, reward, isdone]
    num_states = rows * columns
    isdone = lambda state: state > START_STATE and state <= TERMINAL_STATE
    
    if(isdone(state)):
        next_state = state
    else:
        if(action==0):
            next_state = state-columns if state-columns>=0 else state
        elif(action==1):
            next_state = state+columns if state+columns<num_states else state
        elif(action==2):
            next_state = state+1 if (state+1)%columns else state
        elif(action==3):
            next_state = state-1 if state%columns else state 
    # State Transition Probability is 1 because the environment is deterministic
    return_val = [1, next_state, reward(next_state), isdone(next_state)]
    return return_val

In [4]:
alpha = 0.1 # Learning Rate
epsilon = 0.1 # For Epsilon-greedy policy to balance exploration and exploitation
rows = 4
columns = 12
num_states = rows * columns
num_actions = 4
gamma = 1 # Discount Factor
episodes = 100000 # Number of games played

In [5]:
def sarsa():
    # Initialize the action value function
    Q = np.zeros((num_states, num_actions))
    for episode in range(episodes):
        # Initialize S
        curr_state = START_STATE
        # Pick a random number between 0 and 1
        P = np.random.random()
        if(P > epsilon):
            # Pick the greedy action
            curr_action = np.argmax(Q[curr_state])
        else:
            # Pick a random action to explore
            curr_action = np.random.randint(0, num_actions)
        while True:
            # prob: State Transition Probability 
            # reward, next_state: Immediate reward and next state on taking curr_action in curr_state
            # isdone: Whether the next state is Terminal or not
            prob, next_state, reward, isdone = env(curr_state, curr_action)
            # Pick a random number between 0 and 1
            P = np.random.random()
            if(P > epsilon):
                # Pick the greedy action
                next_action = np.argmax(Q[next_state])
            else:
                # Pick a random action to explore
                next_action = np.random.randint(0, num_actions)
            # Update the current state-action value
            Q[curr_state, curr_action] += alpha * (reward + gamma * Q[next_state, next_action] - Q[curr_state, curr_action])
            curr_state = next_state
            curr_action = next_action
            if isdone:
                break
    return Q

In [5]:
Q = sarsa()
# Deterministic policy obtained using updated Q values
policy = np.argmax(Q,axis=1)
print(f'Value Function for SARSA:\n {Q.reshape(num_states, num_actions)}')
print('\n')
print(f'Deterministic Policy:\n {policy.reshape(rows, columns)}')

Value Function for SARSA:
 [[ -15.94559988  -16.9948888   -14.69603306  -15.72345508]
 [ -15.05465572  -15.76705586  -13.54840985  -15.87710741]
 [ -13.45242819  -14.69868729  -12.16853176  -14.94160782]
 [ -12.63413295  -13.699749    -10.96799573  -13.63458863]
 [ -11.61395304  -13.18976409  -10.12519889  -12.66692852]
 [ -10.21356605  -11.41455096   -8.81595204  -11.54811983]
 [  -8.89894462   -9.44245617   -7.84409062  -10.38606989]
 [  -7.97057786   -7.50709271   -6.86752574   -9.04047756]
 [  -6.71601565   -6.28608529   -5.76857999   -8.10824048]
 [  -5.75416482   -6.05996278   -4.48956925   -6.89134512]
 [  -4.53879864   -3.37399566   -3.69865185   -5.72865438]
 [  -3.29939681   -2.29040333   -3.38761926   -4.6112411 ]
 [ -15.77520247  -18.15055322  -16.08927591  -16.81982097]
 [ -14.82251886  -20.91290962  -15.2901838   -17.36156562]
 [ -13.41420048  -19.27629725  -13.80332536  -15.83643976]
 [ -12.2886318   -14.46490268  -13.37879625  -14.82369556]
 [ -11.34638929  -18.07730175

In [6]:
map_dict = {0:'Up', 1:'Down', 2:'Right', 3:'Left'}

In [7]:
def qlearning():
    # Initialize the action value function
    Q = np.zeros((num_states, num_actions))
    for episode in range(episodes):
        # Initialize S
        curr_state = START_STATE
        while True:
            # Generate a random number between 0 and 1
            P = np.random.random()
            if(P > epsilon):
                # Pick the greedy action
                curr_action = np.argmax(Q[curr_state])
            else:
                # Pick a random action to explore
                curr_action = np.random.randint(0, num_actions)
            # prob: State Transition Probability 
            # reward, next_state: Immediate reward and next state on taking curr_action in curr_state
            # isdone: Whether the next state is Terminal or not    
            prob, next_state, reward, isdone = env(curr_state, curr_action)
            # Update the current state-action value
            Q[curr_state, curr_action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[curr_state, curr_action])
            curr_state = next_state
            if isdone:
                break
    return Q

In [8]:
Q = qlearning()
# Deterministic policy obtained using updated Q values
policy = np.argmax(Q,axis=1)
print(f'Value Function:\n {Q.reshape(num_states, num_actions)}')
print('\n')
print(f'Deterministic Policy:\n {policy.reshape(rows, columns)}')

Value Function:
 [[ -12.98693292  -12.87869316  -12.88008789  -12.93179274]
 [ -12.06615657  -11.96851442  -11.96921728  -11.98307952]
 [ -11.09393501  -10.99036754  -10.98978787  -11.26899692]
 [ -10.15498083   -9.99675241   -9.99668805  -10.6014942 ]
 [  -9.30616091   -8.99841075   -8.99845141   -9.73112232]
 [  -8.43465085   -7.99926849   -7.99926466   -8.13298933]
 [  -7.37140155   -6.99967301   -6.99963809   -8.10063723]
 [  -6.38793972   -5.99984205   -5.99982477   -6.65121566]
 [  -5.32798129   -4.99993813   -4.99993187   -5.71859   ]
 [  -4.47703275   -3.99997724   -3.9999785    -4.93731032]
 [  -3.50419978   -2.99999506   -2.99999465   -3.8717001 ]
 [  -2.52114681   -2.           -2.26217078   -2.49145936]
 [ -13.79299651  -12.          -12.          -12.99977415]
 [ -12.92682326  -11.          -11.          -12.99953489]
 [ -11.96857313  -10.          -10.          -11.99874289]
 [ -10.98998707   -9.           -9.          -10.99791568]
 [  -9.99231524   -8.           -8.    

In [8]:
map_dict = {0:'Up', 1:'Down', 2:'Right', 3:'Left'}

SARSA takes a safer path than Q-learning, which takes the optimal path. 

In [11]:
# Decrease the value of epsilon (exploration rate)
epsilon = 0.001

In [12]:
Q = sarsa()
# Deterministic policy obtained using updated Q values
policy = np.argmax(Q,axis=1)
print(f'Value Function for SARSA:\n {Q.reshape(num_states, num_actions)}')
print('\n')
print(f'Deterministic Policy:\n {policy.reshape(rows, columns)}')

Value Function for SARSA:
 [[-12.11387557 -12.04449439 -12.01281371 -11.97263809]
 [-11.41488169 -11.47062249 -11.42876236 -11.45544762]
 [-10.65272594 -10.60482184 -10.58601944 -10.67164215]
 [ -9.78173365  -9.73059159  -9.70831792  -9.7395596 ]
 [ -8.82396973  -8.79631318  -8.79268092  -8.9510577 ]
 [ -7.86074961  -7.86721059  -7.86580772  -7.87363929]
 [ -7.01043849  -6.9268976   -6.92134178  -6.97949329]
 [ -6.11786885  -5.95194311  -5.95013869  -6.05900709]
 [ -4.97780293  -4.97335312  -4.97042045  -5.1226547 ]
 [ -4.01577536  -3.98903846  -3.98838761  -4.06850761]
 [ -3.06045075  -2.99827569  -2.99826478  -3.01364584]
 [ -2.09632904  -2.00039369  -2.1057925   -2.01546667]
 [-12.64192219 -14.08995187 -12.00769142 -12.94906929]
 [-12.17237884 -12.89960596 -11.00420622 -13.01766285]
 [-11.2010063  -11.73254742 -10.00298998 -11.82863142]
 [-10.2705465  -10.89222334  -9.00454018 -11.09299558]
 [ -9.4881521   -8.4112217   -8.00889838  -9.95036188]
 [ -8.3350533   -7.44934432  -7.014435

SARSA becomes less conservative on decreasing exploration rate